# Collect data from Holzeinschlagsmedung (HEM)

Download link: https://www.verwaltung.steiermark.at/cms/ziel/74835646/DE/, User: 100, Password: 100


In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime as dt
import os
import re

In [ ]:
# All directories

single_HEMs_path = # doesn't exist anymore

output_path      = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023/02 - Data/AUT/"


In [ ]:
AUT_districts_name_corr = {
                 'BFI Imst'                            : 'Imst'                           ,
                 'BFI Innsbruck'                       : 'Innsbruck-Land'                 ,
                 'BFI Kitzb�hel'                      : 'Kitzbühel'                      ,
                 'BFI Kufstein'                        : 'Kufstein'                       ,
                 'BFI Landeck'                         : 'Landeck'                        ,
                 'BFI Osttirol'                        : 'Lienz'                          ,
                 'BFI Reutte'                          : 'Reutte'                         ,
                 'BFI Schwaz'                          : 'Schwaz'                         ,
                 'BFI Steinach'                        : 'Steinach'                       ,
                 'Braunau am Inn'                      : 'Braunau/Inn'                    ,
                 'Bruck und M�dling'                  : 'Bruck/Leitha+Mödling'           ,
                 'Bruck-M�rzzuschlag'                 : 'Bruck/Mur+Mürzzuschlag'         ,
                 'Burgenland Nord'                     : 'BurgenlandNord'                 ,
                 'Burgenland S�d'                     : 'BurgenlandSüd'                  ,
                 'Eisenstadt (Bezirk, Stadt und Rust)' : 'Eisenstadt+Rust'                ,
                 'F�rstenfeld'                        : 'Fürstenfeld'                    ,
                 'Gm�nd und Waidhofen/Thaya'          : 'Gmünd+Waidhofen/Thaya'          ,
                 'G�nserndorf und Mistelbach'         : 'Gänserndorf+Mistelbach'         ,
                 'G�ssing'                            : 'Güssing'                        ,
                 'Hartberg-F�rstenfeld'               : 'Hartberg+Fürstenfeld'           ,
                 'Horn und Hollabrunn'                 : 'Horn+Hollabrunn'                ,
                 'Kirchdorf an der Krems'              : 'Kirchdorf/Krems'                ,
                 'Korneuburg und Tulln'                : 'Korneuburg+Tulln'               ,
                 'K�rnten'                            : 'Kärnten'                        ,
                 'M�rzzuschlag'                       : 'Mürzzuschlag'                   ,
                 'Nieder�sterreich'                   : 'Niederösterreich'               ,
                 'Ober�sterreich'                     : 'Oberösterreich'                 ,
                 'Ried im Innkreis'                    : 'Ried/Innkreis'                  ,
                 'Sch�rding'                          : 'Schärding'                      ,
                 'Spittal an der Drau'                 : 'Spittal/Drau'                   ,
                 'St. Johann'                          : 'St.Johann'                      ,
                 'St. P�lten'                         : 'St.Pölten'                      ,
                 'St. Veit an der Glan'                : 'St.Veit/Glan'                   ,
                 'S�doststeiermark'                   : 'Südoststeiermark'               ,
                 'V�cklabruck'                        : 'Vöcklabruck'                    ,
                 'V�lkermarkt'                        : 'Völkermarkt'                    ,
                 'Wr. Neustadt'                        : 'WienerNeustadt'                 ,
                 'W�rgl (ab 2002 bei Kufstein)'       : 'Wörgl (ab 2002 bei Kufstein)'   ,
                 'Zell am See'                         : 'Zell/See'                       ,
    
                 'Innsbruck (Mag.Abt.VI)'                              : 'Innsbruck-Stadt',
                 'Forstamt Stadtmagistrat Innsbruck'                   : 'Innsbruck-Stadt',
                 'Imst und Silz'                                       : 'Imst'           ,
                 'Kitzb�hel-St.Johann i.T.'                           : 'Kitzbühel'      ,
                 'Kufstein (ab 2002 inkl. W�rgl)'                     : 'Kufstein'       ,
                 'Lienz (ab 2002 inkl. Matrei, ab 2003 inkl. Sillian)' : 'Lienz'          ,
                 'Schwaz (ab 2003 inkl. Zillertal)'                    : 'Schwaz'         ,
                 'Lechtal'                                             : 'Reutte'         }

In [ ]:
list_of_states = {
     'Burgenland'             : 'Burgenland_total'   ,
     'Kärnten'                : 'Carinthia_total'    ,
     'Niederösterreich'       : 'LowerAustria_total' ,
     'Oberösterreich'         : 'UpperAustria_total' ,
     'Steiermark'             : 'Styria_total'       ,
     'Tirol'                  : 'Tyrol_total'        ,
     'Vorarlberg'             : 'Vorarlberg_total'   ,
     'Wien'                   : 'Vienna_total'       ,
}

In [ ]:
# Areas for districts in state of Burgenland to split North and South Burgenland data
Bgld_Nord = {
    'Eisenstadt+Rust' : 42.88 + 453.14 + 20.01,    
    'Mattersburg'     : 237.83,   
    'Neusiedl/See'    : 1038.64,
    'Oberpullendorf'  : 701.44,
}
Bgld_Sued = {   
    'Güssing'         : 485.34,        
    'Jennersdorf'     : 253.34, 
    'Oberwart'        : 732.58,       
}

In [ ]:
# get all csv files inside a specific folder

regexp = re.compile(r'.csv')
csv_files = [path.name for path in os.scandir(single_HEMs_path) if path.is_file() if regexp.search(path.name)]


In [ ]:
# Extract the info from the files that we need and add to HEM dataframe

HEM = pd.DataFrame({'year'                : [],
                    'district'            : [],
                    'beetle_damage (Efm)' : [],
                    'storm_damage (Efm)'  : [] 
                   })

for file in csv_files:

    df = pd.read_csv(dir_path+file, header=None, delimiter=";", nrows=3)
    year     = df.loc[0][0][-4:]
    district = df.loc[1][0]
    
    if df.loc[2][0] != "INSGESAMT":
        print(f"Attention! Problem with file {file}. It's not the TOTAL, but {df.loc[2][0]}. Please check this!")

    df = pd.read_csv(dir_path+file, header=None, delimiter=";", skiprows=4)
    
    
    # Check if we are in the right line (sometimes it changes)
    i_beetle = 28
    
    if (df.loc[i_beetle][0] == "davon Borkenk�fer") and (df.loc[i_beetle-3][0] == "SCHADHOLZ" ) and (df.loc[i_beetle-2][0] == "Nadelholz"):
        pass
    elif (df.loc[i_beetle+4][0] == "davon Borkenk�fer") and (df.loc[i_beetle+4-3][0] == "SCHADHOLZ" ) and (df.loc[i_beetle+4-2][0] == "Nadelholz"):
        i_beetle += 4
    else:
        print(f"Can't find right line in csv. Current line: {i_beetle} in file {file}")
    
 
    i_storm  = i_beetle+3
    if (df.loc[i_storm][0] == "davon Sturm"):
        pass
    else: 
        print(f"Sturmschäden nicht 3 entfernt von BB in file {file}")
    
    beetle_damage = df.loc[i_beetle][1] # Efm
    storm_damage = df.loc[i_storm][1]


    HEM.loc[len(HEM.index)] = (year, district, beetle_damage, storm_damage)



#correct name errors (Umlaute)
HEM['district'] = HEM.district.replace(AUT_districts_name_corr)

# delete dummy values
no_data = ['Auswertung', 'Testbezirk', 'BML',
           'Matrei (ab 2002 bei Lienz)', 'Sillian (ab 2002 bei Lienz)', 'Wörgl (ab 2002 bei Kufstein)', 'Zillertal (ab 2003 bei Schwaz)']
for dummy in no_data:
    HEM.drop(HEM.loc[HEM.district == dummy].index, inplace=True)

    
# Check & Drop duplicates
print(HEM[HEM.duplicated(keep=False)].set_index('district').sort_index().to_string())
HEM.drop_duplicates(subset=['year','district', 'beetle_damage (Efm)', 'storm_damage (Efm)'], inplace=True)

# keep copy of original data in the background
HEM_orig = HEM.copy()


In [ ]:
# Reset modified data to original data
HEM = HEM_orig.copy()

In [ ]:
# CORRECTING inconsistencies with data acquisition


## (1) Merge two districts into one
sum_2subsets = lambda x1, x2: (HEM.loc[HEM.district == x1].set_index('year') + HEM.loc[HEM.district == x2].set_index('year')).reset_index()

mergers = [('Hall', 'Telfs'),
           ('Steinach', 'Innsbruck-Land')]

for merger in mergers:
    print(merger)
    
    if merger[1] == 'Innsbruck-Land':
            HEM.at[HEM.loc[HEM.district == 'Telfs'].index , 'district'] = 'Innsbruck-Land'
        
    for var in ['beetle_damage (Efm)', 'storm_damage (Efm)']:
        
        
        
        HEM.at[HEM.loc[HEM.district == merger[1]].index , var] = sum_2subsets(merger[0],merger[1])[var].values
        
    drop_mask = HEM[HEM.district == merger[0]].index
    HEM.drop(drop_mask, inplace = True)





## (2) Sum up two districts with a third one
sum_3subsets = lambda x1, x2, x3: (HEM.loc[HEM.district == x1].set_index('year') + HEM.loc[HEM.district == x2].set_index('year') + HEM.loc[HEM.district == x3].set_index('year')).reset_index()

subsets = [('Mürzzuschlag', 'Bruck an der Mur', 'Bruck/Mur+Mürzzuschlag'),
           ('Hartberg', 'Fürstenfeld', 'Hartberg+Fürstenfeld'),
           ('Radkersburg', 'Feldbach', 'Südoststeiermark'),
           ('Judenburg', 'Knittelfeld', 'Murtal'),
           ('Stainach', 'Liezen gesamt', 'Liezen')
          ]

for subset in subsets:
    print(subset)
    for var in ['beetle_damage (Efm)', 'storm_damage (Efm)']:
        HEM.at[HEM.loc[HEM.district == subset[2]].index , var] = sum_3subsets(subset[0],subset[1],subset[2])[var].values
    
    drop_mask = HEM[HEM.district.isin(subset[:2])].index
    HEM.drop(drop_mask, inplace = True)


    
    
## (3) Splitting data into two by area
split = lambda x1, var, a,: HEM[HEM.district == x1].set_index('year')[var] * a

for var in ['beetle_damage (Efm)', 'storm_damage (Efm)']:
    HEM.at[HEM.loc[HEM.district == 'Eferding'].index, var] = (HEM[HEM.district == 'Eferding'].set_index('year')[var] + split('Eferding und Wels', var, 0.8498)).values
    HEM.at[HEM.loc[HEM.district == 'Wels'].index, var]     = (HEM[HEM.district == 'Wels'].set_index('year')[var] + split('Eferding und Wels', var, 0.1502)).values

    for ld in Bgld_Nord:
        share_area = Bgld_Nord[ld] / sum(Bgld_Nord.values())
        HEM.at[HEM.loc[HEM.district == ld].index, var] = (HEM[HEM.district == ld].set_index('year')[var] + split('BurgenlandNord', var, share_area)).values

    for ld in Bgld_Sued:
        share_area = Bgld_Sued[ld] / sum(Bgld_Sued.values())
        HEM.at[HEM.loc[HEM.district == ld].index, var] = (HEM[HEM.district == ld].set_index('year')[var] + split('BurgenlandSüd', var, share_area)).values

for drops in ['Eferding und Wels', 'BurgenlandNord', 'BurgenlandSüd']:
    drop_mask = HEM[HEM.district == drops].index
    HEM.drop(drop_mask, inplace = True)


## (4) Summing up two dataframe of another size

def sum_diff_size(df1, df2, var):
    df3 = pd.merge(df1, df2, how='outer', on='year').fillna(0)
    df3[var] = df3[f"{var}_x"] + df3[f"{var}_y"]
    
    return df3[var]

for var in ['beetle_damage (Efm)', 'storm_damage (Efm)']:
    HEM.at[HEM.loc[HEM.district == 'Landeck'].index, var] = sum_diff_size(HEM[HEM.district == 'Landeck'], HEM[HEM.district == 'Ried'], var).values

    drop_mask = HEM[HEM.district == 'Ried'].index
    HEM.drop(drop_mask, inplace = True)

# sort values and replace NaNs
HEM.fillna(0, inplace=True)
HEM = HEM.sort_values(['district', 'year'], ascending=(True, True)).reset_index(drop=True)


In [ ]:
# list of all places
sorted(list(HEM.district.unique()))

In [ ]:
# safe data so far to CSV 
HEM.to_csv(output_path+"Data_BMLRT_LoggingReports", sep=";", index=False)